In [17]:
import sqlite3
import os
import cv2 as cv 
from PIL import Image

## **CREATE TABLES**
--------------------------------------------

In [21]:
def create_table(dirname, filename):
    #set paths and create dir where database is stored
    database_path = os.path.join(os.pardir, dirname, filename)
    database_dir_path = os.path.dirname(database_path)

    if not os.path.exists(database_dir_path):
        os.makedirs(database_path)

    #create database connection
    conn = sqlite3.connect(database_path)
    curs = conn.cursor()

    create_table_statement = """CREATE TABLE if not exists images
                (imageid INTEGER PRIMARY KEY,
                 filepath text not null,
                 filename text not null,
                 resolution text not null
                 );"""
    
    curs.execute(create_table_statement)

    conn.commit()

    conn.close()

In [23]:
create_table("databases", "image_recommender.db")

## **FETCH DATA FROM HARDDRIVE**
-----------------------------------

In [5]:
#check path to harddrive
root_dir = "F:\data\image_data"
if os.path.exists(root_dir):
    print(True)

True


In [10]:
#get resolution
def get_image_size(image_path):
    with Image.open(image_path) as img:
        size= img.size
    return str(size)

In [11]:
#generator
def load_images_from_harddrive(root_dir):
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                size = get_image_size(os.path.join(root, file))
                yield root, file, size

gen = load_images_from_harddrive(root_dir)


## **INSERT DATA INTO DATABASE**
--------------------

In [10]:
# imageid = 0

# for root, file  in load_images_from_harddrive(root_dir):
#     filepath = root
#     filename = file
    
#     curs.execute(f"INSERT INTO IMAGES ('imageid', 'filepath', 'filename') VALUES (?,?, ?)", (imageid, filepath,filename))
#     conn.commit()
#     imageid += 1

# conn.close()

In [14]:
#gen with limit to test
imageid = 0

for _ in range(10):
    root, file, size = next(gen)  
    curs.execute(f"INSERT INTO IMAGES ('imageid', 'filepath', 'filename', 'size') VALUES (?,?,?,?)", (imageid, root, file, size))
    conn.commit()
    imageid += 1

## **TEST IF WORKED**
---------------

In [ ]:
# check if table is created 
print(f"Database created: {os.listdir(database_dir)}")

#show database

def describe_table(curs, table_name):
    curs.execute(f"PRAGMA table_info({table_name})")
    return curs.fetchall()


#check which databases are created
curs.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = curs.fetchall()
print(f"Tables: {tables}")

table_structure = describe_table(curs, "images")
table_structure

In [ ]:
#test to retrieve pictures from database
curs.execute(f"SELECT filepath, filename FROM IMAGES")
firstpic = curs.fetchall()[0]
path = os.path.join(firstpic[0], firstpic[1])
image = Image.open(path)
image.show()

In [13]:
#delete all from table

curs.execute("DELETE FROM IMAGES")
conn.commit()

In [15]:
# select 

curs.execute("SELECT * FROM IMAGES ")
curs.fetchall()

[(0,
  'F:\\data\\image_data\\weather_image_recognition\\dew',
  '2209.jpg',
  '(871, 578)'),
 (1,
  'F:\\data\\image_data\\weather_image_recognition\\dew',
  '2210.jpg',
  '(806, 564)'),
 (2,
  'F:\\data\\image_data\\weather_image_recognition\\dew',
  '2211.jpg',
  '(1200, 797)'),
 (3,
  'F:\\data\\image_data\\weather_image_recognition\\dew',
  '2212.jpg',
  '(947, 592)'),
 (4,
  'F:\\data\\image_data\\weather_image_recognition\\dew',
  '2213.jpg',
  '(500, 411)'),
 (5,
  'F:\\data\\image_data\\weather_image_recognition\\dew',
  '2214.jpg',
  '(777, 518)'),
 (6,
  'F:\\data\\image_data\\weather_image_recognition\\dew',
  '2215.jpg',
  '(800, 500)'),
 (7,
  'F:\\data\\image_data\\weather_image_recognition\\dew',
  '2216.jpg',
  '(500, 750)'),
 (8,
  'F:\\data\\image_data\\weather_image_recognition\\dew',
  '2217.jpg',
  '(752, 500)'),
 (9,
  'F:\\data\\image_data\\weather_image_recognition\\dew',
  '2218.jpg',
  '(688, 516)')]

In [ ]:
#delete table if needed 
table_name = ["images", "measures"]

for name in table_name:
    curs.execute(f"DROP TABLE if exists {name}")